## Implement Secure Remote Password (SRP)

To understand SRP, look at how you generate an AES key from DH; now, just observe you can do the "opposite" operation and generate a numeric parameter from a hash. Then:

Replace A and B with C and S (client & server)
```
C & S
    Agree on N=[NIST Prime], g=2, k=3, I (email), P (password)
S

        Generate salt as random integer
        Generate string xH=SHA256(salt|password)
        Convert xH to integer x somehow (put 0x on hexdigest)
        Generate v=g**x % N
        Save everything but x, xH

C->S
    Send I, A=g**a % N (a la Diffie Hellman)
S->C
    Send salt, B=kv + g**b % N
S, C
    Compute string uH = SHA256(A|B), u = integer of uH
C

        Generate string xH=SHA256(salt|password)
        Convert xH to integer x somehow (put 0x on hexdigest)
        Generate S = (B - k * g**x)**(a + u * x) % N
        Generate K = SHA256(S)

S

        Generate S = (A * v**u) ** b % N
        Generate K = SHA256(S)

C->S
    Send HMAC-SHA256(K, salt)
S->C
    Send "OK" if HMAC-SHA256(K, salt) validates
```

You're going to want to do this at a REPL of some sort; it may take a couple tries.

It doesn't matter how you go from integer to string or string to integer (where things are going in or out of SHA256) as long as you do it consistently. I tested by using the ASCII decimal representation of integers as input to SHA256, and by converting the hexdigest to an integer when processing its output.

This is basically Diffie Hellman with a tweak of mixing the password into the public keys. The server also takes an extra step to avoid storing an easily crackable password-equivalent.


## Parameter negotiation
Carol and Steve agree on $N$, $g$, and $k$.

In [1]:
N = int(
"ffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024"
"e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd"
"3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec"
"6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f"
"24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361"
"c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552"
"bb9ed529077096966d670c354e4abc9804f1746c08ca237327fff"
"fffffffffffff", 16)
N

2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919

Humm, curious bit length.

In [10]:
N.bit_length()

1536

In [2]:
g = 2
k = 3

In [5]:
import hashlib 
import os
from Crypto.Util.number import long_to_bytes, bytes_to_long

In [4]:
I = b'carol'
P = b'foobar'

## Client registration
To establish a password with Steve, Carol picks a random salt $s$ and calculates $x = H(s, P)$ for a verifier $v = g^x\mod{N}$.

Carol discards $x$ and sends $I$, $s$, and $v$ to Steve.

Steve stores $I$, $s$, and $v$.

In [6]:
s = os.urandom(8)
s.hex()

'f47581a94888a8aa'

In [7]:
def client_x(salt, password):
    h = hashlib.sha256()
    h.update(salt)
    h.update(password)
    return bytes_to_long(h.digest())

In [8]:
def verifier(salt, password):
    x = client_x(salt, password)
    return pow(g, x, N)

In [9]:
v = verifier(s, P)
v

1650349257589804035174719337388500361604207731238101840288664779903838041600484675975330511755236522208860753321193745160912849072809972474437260178373125679338131337030425662929828322737887250063773156713001187013299018904465792728275494379397949016069509061056894014256974453269645787142741514828758709477751310711946561510153613306689726811718763376346217584665376104242194721055051859112105186686863449652174099885208769011538382887287064133857532726291629629

## Proof of password
Carol sends $I$ and $A = g^a\mod{N}$.

In [9]:
a = randrange(N)
A = pow(g, a, N)

Steve sends salt $s$ and $B = kv + g^b\mod{N}$.

In [10]:
b = randrange(N)
B = k * v + pow(g, b, N)
B = B % N

In [11]:
def scramble(A, B):
    h = hashlib.sha256()
    h.update(long_to_bytes(A))
    h.update(long_to_bytes(B))
    return bytes_to_long(h.digest())

Random scrambling parameter $u = sha256(A|B)$.

In [12]:
u = scramble(A, B)
u

44688017366769106269027601439288373314921681036842707310469132354182986771804

In [13]:
x = client_x(s, P)
x

23600815288738584080465857256312575424438424966955075886375505177003911499397

Carol calculates $S = (B - kg^x)^{a + ux}\mod{N}$.

In [19]:
S_c = pow(B - k * pow(g, x, N), a + u * x, N)
K_c = hashlib.sha256(long_to_bytes(S_c)).digest()

Steve calculates $S = (Av^u)^b\mod{N}$.

In [20]:
S_s = pow(A * pow(v, u, N), b, N)
K_s = hashlib.sha256(long_to_bytes(S_s)).digest()

In [16]:
import hmac

Prove to each other that their keys match.

Carol sends $hmac(K_c, s)$.

In [17]:
mac = hmac.digest(K_c, long_to_bytes(s), 'sha256')

Steve verifies with $hmac(K_s, s)$.

In [18]:
hmac.compare_digest(mac, hmac.digest(K_s, long_to_bytes(s), 'sha256'))

True

_How does a keyed hash (MAC) over the salt compare to the two other methods described in wikipedia entry [18]?_